## test
经测试，发现如下情况：<br>
1、typescript语言下有common,tsx和typescript三个文件，其中common和tsx均可以通过测试，但是typescript无法通过测试。<br>
2、sql:
```
SELECT x -> '$';


[['sql_stmt_list', ['sql_stmt', ['select_stmt', ['SELECT'], ['blob_literal', ['ERROR']]]]]]


(sql_stmt_list
  (sql_stmt
    (select_stmt
      (SELECT)
      (identifier)
      (string_literal))))

[['sql_stmt_list', ['sql_stmt', ['select_stmt', ['SELECT'], ['identifier'], ['string_literal']]]]]
```
3、ruby:
```
puts













"hi"


[['program', ['identifier'], ['string', ['string_content']]]]


(program (call (identifier) (argument_list (string (string_content)))))

[['program', ['call', ['identifier'], ['argument_list', ['string', ['string_content']]]]]]
```
4、Pascal：
```
{$mode delphi}
{$define foo}
{$ifdef foo}
{$m+}
{$I source.inc}
{$endif}


[['root', ['pp'], ['pp'], ['pp'], ['pp'], ['pp'], ['pp'], ['declVars', ['MISSING']]]]


(root
   (pp)
   (pp)
   (pp)
   (pp)
   (pp)
   (pp))


[['root', ['pp'], ['pp'], ['pp'], ['pp'], ['pp'], ['pp']]]
```
5、lua语言print函数输出都有问题：以下是其中一个例子
```



print('')
print("")
print('"You\'re a wizard, Harry!"')
print("\"I'm a what?\"")
print('"Did you put your name into the Goblet of Fire, Harry?" -- Dumbledore asked calmly.')


[['chunk', ['ERROR', ['variable', ['identifier']], ['parenthesized_expression', ['string']]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]]]]


(chunk
  (call
    function: (variable
      name: (identifier))
    arguments: (argument_list
      (expression_list
        (string))))
  (call
    function: (variable
      name: (identifier))
    arguments: (argument_list
      (expression_list
        (string))))
  (call
    function: (variable
      name: (identifier))
    arguments: (argument_list
      (expression_list
        (string))))
  (call
    function: (variable
      name: (identifier))
    arguments: (argument_list
      (expression_list
        (string))))
  (call
    function: (variable
      name: (identifier))
    arguments: (argument_list
      (expression_list
        (string)))))


[['chunk', ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]], ['call', ['variable', ['identifier']], ['argument_list', ['expression_list', ['string']]]]]]
```


In [59]:
import re
from tree_sitter import Language, Parser
from ipynb.fs.full.setting import *#ipynb模块，引用setting.ipynb函数
#获得代码的lisp风格ast
def getast(language,text):
    LANGUAGE=Language('build/my-languages.so', language)
    parser=Parser()
    parser.set_language(LANGUAGE)
    tree=parser.parse(bytes(text,"utf8"))
    root=tree.root_node
    return root.sexp()
#对lisp风格的ast生成的列表进行递归，去除如'left:''right:'等多余的标志符（测试文件的ast和解析生成的ast略有区别）
def recurse(mylist):
    newlist=[]
    for i in range(len(mylist)):
        if type(mylist[i])==list:
            newlist.append(recurse(mylist[i]))
        else:
            if i>0 and type(mylist[i-1])==str:
                pass
            elif i>0 and type(mylist[i+1])==list and type(mylist[i-1])==list:
                pass
            else:
               newlist.append(mylist[i]) 
    return newlist
#主处理函数
def process(language,text):
    code=[]
    ast=[]
    text=re.split(r'={3,}\n',text)
    #根据源测试文件的格式对文件进行分割
    for i in text:
        if re.search(r'-{3,}',i)!=None:
            i1=re.split(r'-{3,}',i)
            code.append(i1[0])
            ast.append(i1[1])
    if language=='c-sharp':
        language='c_sharp'
    elif language=='typescript':
        language='tsx'
    count=0;
    for i in range(len(code)):
        #生成lisp风格的ast
        string=getast(language,code[i])
        string=string.replace('(','[')
        string=string.replace(')',']')
        #将lisp风格的ast转化为python列表
        list=process_string(string)
        

        #对测试文件中的lisp风格ast进行处理，转化为python列表
        string1=ast[i].replace('\n','')
        string1=string1.replace('\t','')
        string1=string1.replace('(','[')
        string1=string1.replace(')',']')
        list1=process_string(string1)
        #去除掉双方多余的标志符。
        list=recurse(list)
        list1=recurse(list1)
        #提示错误信息
        if list!=list1:
            count+=1
            print(code[i])
            print(list)
            print(ast[i])
            print(list1)
            print('wrong\n\n')
    if count==0:
        print('Pass the test\n')
    



In [60]:
import os
file="./vendor"
#遍历文件，进行测试
for root,dirs,files,in os.walk(file):
    string=root.split('\\')
    if len(string)>1:
        language=string[1].split('tree-sitter-')
        language=language[1]
    if string[-1]=='corpus':
        for roots, dirs, files in os.walk(root):
            for files1 in files:
                path=os.path.join(root, files1)            
                with open(path,"r") as f:
                    text=f.read()
                    print(path+"\n")
                    process(language,text)

./vendor\tree-sitter-c\test\corpus\ambiguities.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\crlf.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\declarations.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\expressions.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\microsoft.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\preprocessor.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\statements.txt

Pass the test

./vendor\tree-sitter-c\test\corpus\types.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\attributes.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\classes.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\contextual-keywords.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\enums.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\expressions.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\identifiers.txt

Pass the test

./vendor\tree-sitter-c-sharp\corpus\interfaces.txt

Pass the test

./vend